In [1]:
from molmap import model as molmodel
import molmap

import matplotlib.pyplot as plt

import pandas as pd
from tqdm import tqdm
from joblib import load, dump
tqdm.pandas(ascii=True)
import numpy as np

import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
np.random.seed(7)
tf.compat.v1.set_random_seed(7)


RDKit WARNING: [00:10:04] Enabling RDKit 2019.09.2 jupyter extensions


In [2]:
tf.__version__

'2.0.0'

In [3]:
tmp_feature_dir = './tmpignore'
if not os.path.exists(tmp_feature_dir):
    os.makedirs(tmp_feature_dir)

In [4]:
#load dataset
from molmap import dataset
data = dataset.load_CYP450()
df = data.data
X_smiles = df.smiles.tolist()
task_name = 'CYP450'


total samples: 16896


In [5]:
MASK = -1
tasks = ['label_1a2', 'label_2c19', 'label_2c9', 'label_2d6', 'label_3a4']
Y = df[tasks].astype('float').fillna(MASK).values
if Y.shape[1] == 0:
    Y = Y.reshape(-1, 1)
Y.shape

(16896, 5)

# featurizer

In [6]:
mp1 = molmap.loadmap('../descriptor.mp')
mp2 = molmap.loadmap('../fingerprint.mp')

X1_name = os.path.join(tmp_feature_dir, 'X1_%s.data' % task_name)
X2_name = os.path.join(tmp_feature_dir, 'X2_%s.data' % task_name)
if not os.path.exists(X1_name):
    X1 = mp1.batch_transform(df.smiles, n_jobs = 8)
    dump(X1, X1_name)
else:
    X1 = load(X1_name)

if not os.path.exists(X2_name): 
    X2 = mp2.batch_transform(df.smiles, n_jobs = 8)
    dump(X2, X2_name)
else:
    X2 = load(X2_name)

molmap1_size = X1.shape[1:]
molmap2_size = X2.shape[1:]

# Perfor a 10 fold Cross-validation to find best epochs

In [7]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, shuffle=True, random_state=123)

train_valid_idx = df[df.group != 'test set'].index.tolist()
test_idx = df[df.group == 'test set'].index.tolist()
testX = (X1[test_idx], X2[test_idx])
testY = Y[test_idx]

def get_pos_weights(trainY):
    """pos_weights: neg_n / pos_n """
    dfY = pd.DataFrame(trainY)
    pos = dfY == 1
    pos_n = pos.sum(axis=0)
    neg = dfY == 0
    neg_n = neg.sum(axis=0)
    pos_weights = (neg_n / pos_n).values
    neg_weights = (pos_n / neg_n).values
    return pos_weights, neg_weights

pos_weights, neg_weights = get_pos_weights(Y[train_valid_idx])

epochs = 800
patience = 10 #early stopping

dense_layers = [256, 128, 32]
batch_size = 128
lr = 1e-4
weight_decay = 0
monitor = 'val_auc'
metric = 'ROC'
dense_avf = 'relu'
last_avf = None #sigmoid in loss

## 10 fold-cv performance

In [8]:
import tensorflow_addons as tfa

i = 0
for train_idx, valid_idx in kf.split(train_valid_idx):
    
    filename = 'model-fold-%s.csv' % str(i).zfill(2)
    
    if os.path.exists(filename):
        continue
        
    trainX = (X1[train_idx], X2[train_idx])
    trainY = Y[train_idx]

    validX = (X1[valid_idx], X2[valid_idx])
    validY = Y[valid_idx]

    loss = lambda y_true, y_pred: molmodel.loss.weighted_cross_entropy(y_true,y_pred, pos_weights, MASK = -1)
    opt = tfa.optimizers.AdamW(weight_decay = 0.0,learning_rate=lr,beta_1=0.9,beta_2=0.999, epsilon=1e-08)

    model = molmodel.net.DoublePathNet(molmap1_size, molmap2_size, 
                                       n_outputs=Y.shape[-1], 
                                       dense_layers=dense_layers, 
                                       dense_avf = dense_avf, 
                                       last_avf=last_avf)
    model.compile(optimizer = opt, loss = loss)
    performance = molmodel.cbks.CLA_EarlyStoppingAndPerformance((trainX, trainY), 
                                                                (validX, validY), 
                                                                patience = patience,
                                                                metric = metric,
                                                                criteria = monitor)

    model.fit(trainX, trainY, batch_size=batch_size, 
              epochs=epochs, verbose= 0, shuffle = True, 
              validation_data = (validX, validY), 
              callbacks=[performance]) 


    best_epoch = performance.best_epoch
    train_pfs = performance.evaluate(trainX, trainY)            
    valid_pfs = performance.evaluate(validX, validY)            
    final_res = {    'fold':i,
                     'task_name':task_name,            
                     'train_pfs':train_pfs, 
                     'metric': metric,
                     'valid_pfs':valid_pfs,                      
                     'best_epoch': best_epoch,
                     'batch_size':batch_size,
                     'trainable_params': model.count_params(),
                     'lr': lr,
                     'weight_decay':weight_decay
                    }
    
    
    pd.DataFrame([final_res]).to_csv(filename)

    i += 1

### training the final model using the same Hyperparameters, and test on the external dataset

In [9]:
from glob import glob

In [10]:
csvs = glob('./model-*.csv')
dfp = pd.concat([pd.read_csv(i, index_col = 0) for i in csvs])
dfp = dfp.sort_values('fold')
dfp

,fold,task_name,train_pfs,metric,valid_pfs,best_epoch,batch_size,trainable_params,lr,weight_decay
0,0,CYP450,"[0.9874567195572976, 0.9738906675505736, 0.990...",ROC,"[0.9490147443847321, 0.8935858711631306, 0.908...",61,128,803813,0.0001,0
0,1,CYP450,"[0.9741909012094102, 0.9501959277403189, 0.976...",ROC,"[0.9507553234977144, 0.9131923439881918, 0.927...",46,128,803813,0.0001,0
0,2,CYP450,"[0.9794194858383412, 0.9712625715859213, 0.985...",ROC,"[0.9456466250709019, 0.911163739385852, 0.9278...",56,128,803813,0.0001,0
0,3,CYP450,"[0.9624161673656005, 0.936330747601789, 0.9508...",ROC,"[0.9486500259610392, 0.8955121766022572, 0.908...",30,128,803813,0.0001,0
0,4,CYP450,"[0.9695334924093093, 0.9466458384578187, 0.965...",ROC,"[0.9438940878882017, 0.8930659049511509, 0.910...",41,128,803813,0.0001,0
0,5,CYP450,"[0.9759679586157101, 0.9586982189110491, 0.975...",ROC,"[0.944468861249066, 0.9071703424840086, 0.9149...",46,128,803813,0.0001,0
0,6,CYP450,"[0.9817337929595712, 0.9629797182474445, 0.977...",ROC,"[0.9375272350933257, 0.9001070181367579, 0.922...",53,128,803813,0.0001,0
0,7,CYP450,"[0.9839607916934181, 0.9674159823631325, 0.987...",ROC,"[0.946979019942175, 0.8919884270651575, 0.9216...",54,128,803813,0.0001,0
0,8,CYP450,"[0.9815696344422685, 0.9640635372888541, 0.980...",ROC,"[0.9383434758168977, 0.9018265212849308, 0.920...",51,128,803813,0.0001,0
0,9,CYP450,"[0.9860715012616572, 0.9789726263249825, 0.991...",ROC,"[0.9489664831449822, 0.8941442162924048, 0.912...",61,128,803813,0.0001,0


In [11]:
dfp.valid_pfs = dfp.valid_pfs.apply(lambda x:[float(i.replace('[','').replace(']','')) for i in x.split(',')])

In [12]:
cv_performances = pd.DataFrame(dfp.valid_pfs.to_list(), columns= tasks)
cv_performances.to_csv('10foldcsv_auc.csv')
cv_performances

,label_1a2,label_2c19,label_2c9,label_2d6,label_3a4
0,0.949015,0.893586,0.908405,0.885345,0.908443
1,0.950755,0.913192,0.927266,0.878216,0.923830
2,0.945647,0.911164,0.927832,0.887480,0.916079
3,0.948650,0.895512,0.908302,0.888027,0.908995
4,0.943894,0.893066,0.910136,0.899558,0.907428
5,0.944469,0.907170,0.914973,0.883796,0.896696
6,0.937527,0.900107,0.922658,0.906264,0.911466
7,0.946979,0.891988,0.921601,0.893148,0.915804
8,0.938343,0.901827,0.920993,0.889568,0.923266
9,0.948966,0.894144,0.912266,0.877357,0.920157


In [13]:
cv_performances.mean().round(3)

label_1a2     0.945
label_2c19    0.900
label_2c9     0.917
label_2d6     0.889
label_3a4     0.913
dtype: float64

In [14]:
cv_performances.std().round(3)

label_1a2     0.005
label_2c19    0.008
label_2c9     0.008
label_2d6     0.009
label_3a4     0.008
dtype: float64

In [15]:
best_epoch = dfp.best_epoch.mode().iloc[0]

In [16]:
trainX = (X1[train_valid_idx], X2[train_valid_idx])
trainY = Y[train_valid_idx]

loss = lambda y_true, y_pred: molmodel.loss.weighted_cross_entropy(y_true,y_pred, pos_weights, MASK = -1)
opt = tf.keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) #

model = molmodel.net.DoublePathNet(molmap1_size, molmap2_size, 
                                   n_outputs=Y.shape[-1], 
                                   dense_layers=dense_layers, 
                                   dense_avf = dense_avf, 
                                   last_avf=last_avf)
model.compile(optimizer = opt, loss = loss)

performance = molmodel.cbks.CLA_EarlyStoppingAndPerformance((trainX, trainY), 
                                                            (testX, testY), 
                                                            patience = 1000000,
                                                            metric = metric,
                                                            criteria = monitor)

model.fit(trainX, trainY, batch_size=batch_size, 
          epochs=best_epoch, verbose= 0, shuffle = True, 
          validation_data = (testX, testY), 
          callbacks=[performance]) 


epoch: 0001, loss: 0.8188 - val_loss: 0.6775; auc: 0.7976 - val_auc: 0.7831                                                                                                    
epoch: 0002, loss: 0.7191 - val_loss: 0.6578; auc: 0.8512 - val_auc: 0.8245                                                                                                    
epoch: 0003, loss: 0.6716 - val_loss: 0.6498; auc: 0.8698 - val_auc: 0.8368                                                                                                    
epoch: 0004, loss: 0.6517 - val_loss: 0.6472; auc: 0.8791 - val_auc: 0.8426                                                                                                    
epoch: 0005, loss: 0.6373 - val_loss: 0.6524; auc: 0.8851 - val_auc: 0.8448                                                                                                    
epoch: 0006, loss: 0.6304 - val_loss: 0.6563; auc: 0.8899 - val_auc: 0.8481                                             

In [17]:
performance.evaluate(testX, testY) # test roc_auc

[0.9746272458316189,
 0.8042402039654447,
 0.8195457640307363,
 0.8997232275920801,
 0.9213991020507221]

In [18]:
np.mean(performance.evaluate(testX, testY)) # test roc_auc

0.8839071086941204